#**Analysis of a Dead Cat Bounce for the stock market with focus on the S&P 500**




# **Business Understanding**

**What is our main goal with this project?**


> Offer possibilities for short-term trades.

> Indicate false recovery and alert long-term trades to not invest. 



# **Data Preperation**

Installing all necessary dependencies for the project. Packages for RFT are listed seperately.



In [ ]:
#Install the dependencies
import numpy as np 
import pandas as pd
import os, io
import plotly.graph_objs as go
import datetime as dt
import requests
import pandas_datareader.data as web
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt



---


*   Main usage of yahoo finance API.
*   Working with the S&P500 stock.
*   Data is generated automatically.
*   Time period flexible.



---

The S&P500 index consists of the 500 most valuable companies from the USA and combines them in an index. Therefore specific market movements are more average. It helps to train the model faster and better.

In [ ]:
stock_id = 'PLUG'

start = dt.datetime(2019,1,1)
end = dt.datetime.now()
end_test = dt.datetime(2020,3,15)

stock = web.DataReader(stock_id, 'yahoo', start, end)
df = stock
df.dropna(inplace=True)

NameError: ignored

**Data Modelling for:**


*   Moving Averages (EMA)
*   MACD
*   Signalline
*   Histogram
*   Bollinger Bands
*   Decision Tree
*   Dead Cat Bounce



In [ ]:
indicator = pd.DataFrame(df, index=df.index)

# 30 day
short_window = 30
# 50 day
long_window = 50
# 20 day
bollinger_avg = 20

# Exponential moving averages using the closing data
indicator['short_avg'] = df['Close'].ewm(span=short_window, adjust=False).mean()
indicator['long_avg'] = df['Close'].ewm(span=long_window, adjust=False).mean()

# Defining bollinger averages for 20 days.
indicator['bollinger_avg'] = df['Close'].ewm(span=bollinger_avg, adjust=False).mean()

# Compute EMA by defining short- and long average. 12 days and 26 days
indicator['ema_short_avg'] = df['Close'].ewm(span=12, adjust=False).mean()
indicator['ema_long_avg'] = df['Close'].ewm(span=26, adjust=False).mean()

# Get MACD by dividing EMA (12 days) - EMA (26 days)
indicator['macd'] = indicator['ema_short_avg'] - indicator['ema_long_avg']

# Another parameter is the signal line with a smoothing of nine days, which is calculated as a moving average of the MACD line.
indicator['signalline'] = indicator['macd'].ewm(span=9, adjust=False).mean()

# Used to project histogram and identify falling or rising movements. Important to correctly analyze a Dead-Cat-Bounce
# Info: *2 is used for better visualization
indicator['histogram_macd_signalline'] = (indicator['macd'] - indicator['signalline'])*2

# Calculate 30 Day Moving Average, Std Deviation, Upper Band and Lower Band
indicator['short_avg_ma'] = df['Close'].rolling(window=bollinger_avg).mean()

# Calculate short average standard deviation for plotting bollinger (avg) bands
indicator['short_avg_std'] = df['Close'].rolling(window=bollinger_avg).std()

# calculate upper and lower band 
indicator['upper_band'] = indicator['short_avg_ma'] + (indicator['short_avg_std'] *2)
indicator['lower_band'] = indicator['short_avg_ma'] - (indicator['short_avg_std'] *2)

# **Telegram Bot Setup**

In [ ]:
def send_push(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = 'token'
    bot_chat_id = 'chat'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

# **Method Gathering (mainly for plots)**

This section is used to build methods which will support further reading.

In [ ]:
# Build methods for plotting more easily graphs
def figure_add_trace(figure, indicator, name, color):
  figure.add_trace(go.Scatter(x=df.index,
                              y=indicator,
                              mode='lines',
                              name=name,
                              line=go.scatter.Line(color=color)))
  
def figure_add_fill_trace(figure, indicator, fill, name, color):
  figure.add_trace(go.Scatter(x=df.index,
                              y=indicator,
                              fill=fill,
                              name=name,
                              mode='lines',
                              fillcolor='rgba(220,220,220,0.5)',
                              line_color=color))

def figure_update_layout(figure, name, xaxis_label, yaxis_label):
  figure.update_layout(
      title=name, 
      xaxis_title=xaxis_label,
      yaxis_title=yaxis_label,
      xaxis_rangeslider_visible=False,
      )

# **Moving Averages & Bollinger Bands**

**What is a Moving Average (MA)?**


> In statistics, a moving average is a calculation used to analyze data points by creating a series of averages of different subsets of the full data set.

> The reason for calculating the moving average of a stock is to help smooth out the price data by creating a constantly updated average price.

> Moving averages are usually calculated to identify the trend direction of a stock or to determine its support and resistance levels. It is a trend-following—or lagging—indicator because it is based on past prices.



---


**What are the so called "Bollinger Bands"?**


> A Bollinger Band is a technical analysis tool defined by a set of trendlines plotted two standard deviations (positively and negatively) away from a simple moving average (SMA) of a security's price, but which can be adjusted to user preferences.

> Many traders believe the closer the prices move to the upper band, the more overbought the market, and the closer the prices move to the lower band, the more oversold the market.

> The Bollinger Bands basically create a dynamic trend, as the moving average moves continuously and follows the movements of the underlying asset. Therefore, in volatile times, the spacing of the Bollinger Bands is relatively wide and in calm market phases, it is much smaller.





In [ ]:
# S&P500 Stock
figure_stock = go.Figure(data=[
                               go.Candlestick(
                                   x=df.index,
                                       open=df['Open'],
                                       high=df['High'],
                                       low=df['Low'],
                                       close=df['Close'],
                                       name='S&P 500'),
                               ])

In [ ]:
figure_add_fill_trace(figure_stock, indicator['upper_band'], None, 'Lower Band', 'lightgrey')
figure_add_fill_trace(figure_stock, indicator['lower_band'], 'tonexty', 'Upper Band', 'lightgrey')
figure_add_trace(figure_stock, indicator['bollinger_avg'], 'Bollinger Average', 'black')

figure_add_trace(figure_stock, indicator['short_avg'], '30 Days Average', 'yellow')
figure_add_trace(figure_stock, indicator['long_avg'], '50 Days Average', 'purple')
figure_update_layout(figure_stock, 'Stock Movement for S&P500', 'Date', 'Stock Price')

# **MACD & Signalline**

**What is the MACD & the Signalline?**


The MACD turns two trend-following indicator, moving averages, into a momentum oscillator by subtracting the longer moving average from the shorter one. As a result, the MACD offers trend following and momentum for the stock market.
The MACD fluctuates above and below the zero line as the moving averages converge, cross and diverge. MACD is all about the convergence and divergence of the two moving averages.



---
Calculation:
> MACD Line: (12-day EMA - 26-day EMA): Time period is flexible and depends on trading style.

> Signal Line: 9-day EMA of MACD Line

> MACD Histogram: MACD Line - Signal Line


---
Why should we use the MACD for understanding the Dead-Cat-Bounce?
> If Signal-Line is bigger than MACD, the market tends to fall.

> Dead-Cat-Bounce spotable by plotting histogram.

> Crossovers represent a change in market movement. 






In [ ]:
# Graph with MACD, Signalline and Histrogram
figure_macd = go.Figure()
figure_add_trace(figure_macd, indicator['macd'], 'MACD', 'blue')
figure_add_trace(figure_macd, indicator['signalline'], 'Signal-Line (SL)', 'red')

figure_macd.add_trace(go.Bar(x=df.index,
                        y=indicator['histogram_macd_signalline'],
                        name='MACD - SL',
                        textposition='auto',
                        marker_color='lightslategray'))

figure_update_layout(figure_macd, 'MACD and Signal-Line for S&P500', 'Date', '')

# **Dead Cat Bounce Calculation**

**What conditions should be considered to analyze a dead-cat-bounce?**


> Price is already in a downward trend (bear market).

> A Dead Cat Bounce occurs when the price tips down 5% or more, continues to fall, but then experiences a rally.



In [ ]:
def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + t           

    avg = sum_num / len(num)
    return avg

pct_change_avg = cal_average([df['Close'], df['Open']])
pct_change_avg

In [ ]:
set_freq = '2D'

pct_change_close = df['Close'].pct_change(freq=set_freq, fill_method='ffill')
pct_change_open = df['Open'].pct_change(freq=set_freq, fill_method='ffill')

pct_change_close.dropna(inplace=True)
pct_change_open.dropna(inplace=True)

pct_change = pct_change_avg.pct_change(freq=set_freq, fill_method='ffill')

dead_cat_bounce = np.where((indicator.signalline > indicator.macd) &
                          ((pct_change >= 0.05) | (pct_change <= -0.05)) &
                          ((df.Open < indicator.lower_band) | (df.Close < indicator.lower_band)),
                          True, False)

indicator['dead_cat_bounce'] = dead_cat_bounce

dead_cat_bounce = indicator[(indicator['dead_cat_bounce'] == True)]

In [ ]:
# Plotting historic dead cat bounce is intended to show the supervisor
# that everything is working properly.

def plot_dead_cat_bounce():
  figure_stock.add_trace(
      go.Scatter(
          mode='markers',
          x=dead_cat_bounce.index,
          y=dead_cat_bounce['Open'],
          marker_symbol='triangle-down',
          marker=dict(
              color='Orange',
              size=20,
              opacity=0.5,
              line=dict(
                  color='Red',
                  width=2)),
          showlegend=True,
          name='Dead Cat Bounce Without Prediction'))

# **DECISION-TREE PREDICTION MODEL**



> The Decision Tree is used to forecast the movement of the stock market. By giving the odel historical Data as an Input it can Build a Tree and predict Future prizes. The Model is using Knots which are connected with branches which are the different decisions. The solutions are shown as the leaves.

> Compare to source: https://randerson112358.medium.com/predict-stock-prices-using-machine-learning-python-f554b7167b36


In [ ]:
df_rf = stock
df_rf.dropna(inplace=True)

In [ ]:
df_prediction = stock

#Create a variable to predict 'x' days out into the future
future_days = 25

#Create a new column (the target or dependent variable) shifted 'x' units/days up
df_prediction['Prediction'] = df_prediction[['Close']].shift(-future_days)

In [ ]:
X = np.array(df_prediction.drop(['Prediction'], 1))[:-future_days]

In [ ]:
y = np.array(df_prediction['Prediction'])[:-future_days]

In [ ]:
#Splitting the Date (25% Testdata/75% Trainingdata)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)


In [ ]:
#Create the decision tree regressor model
tree = DecisionTreeRegressor().fit(x_train, y_train)

In [ ]:
#Get the data of the future
x_future = df_prediction.drop(['Prediction'], 1)[:-future_days]

#Get the last rows
x_future = x_future.tail(future_days) 

#Convert the data set into a numpy array for using it with the prediction
x_future = np.array(x_future)

#Set the model tree prediction
tree_prediction = tree.predict(x_future)

In [ ]:
#Plot the data
valid =  df_prediction[X.shape[0]:]
valid['Prediction'] = tree_prediction 
  
# S&P500 Stock
figure_prediction = go.Figure(data=[
                               go.Candlestick(
                                   x=df.index,
                                       open=df['Open'],
                                       high=df['High'],
                                       low=df['Low'],
                                       close=df['Close'],
                                       name='S&P 500'),
                               ])

figure_prediction.add_trace(go.Scatter(x=valid.index,
                              y=valid['Prediction'],
                              mode='lines',
                              name='Prediction',
                              line=go.scatter.Line(color='blue')))

figure_update_layout(figure_prediction, 'Prediction - Stock Movement', 'Date', 'Stock Price')

In [ ]:
#Calculate the Mean devation
valid['devation']=valid['Close']-valid['Prediction']
mean_devation=valid['devation'].sum()/valid['devation'].count()

In [ ]:
#Calculating the accuracy 
accuracy=1-(valid['devation']!=0).sum()/valid['devation'].count()
print('Accuracy:\t\t\t',accuracy*100,'%\nDurchschnittliche Abweichung:\t',mean_devation)

# **Check future days and if the market tends to fall**

Predict future trend movements by calculating MACD and Signalline.

In [ ]:
#MACD for future days
predict_macd = pd.DataFrame(df, index=df.index)

# Compute EMA by defining short- and long average. 12 days and 26 days
predict_macd['ema_short_avg'] = valid['Close'].ewm(span=12, adjust=False).mean()
predict_macd['ema_long_avg'] = valid['Close'].ewm(span=26, adjust=False).mean()

# Get MACD by dividing EMA (12 days) - EMA (26 days)
predict_macd['macd'] = predict_macd['ema_short_avg'] - predict_macd['ema_long_avg']

predict_macd['signalline'] = predict_macd['macd'].ewm(span=9, adjust=False).mean()

predict_macd.dropna(inplace=True)

In [ ]:
print(predict_macd['signalline'].mean(), predict_macd['macd'].mean())

In [ ]:
def predict_falling_market():
  future_dead_cat_bounce = np.where((predict_macd['signalline'].mean() > predict_macd['macd'].mean()) &
                                  (indicator.signalline > indicator.macd) &
                                  ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                                  (indicator.Open < indicator.lower_band),
                                  True, False)

  indicator['future_dead_cat_bounce'] = future_dead_cat_bounce

  future_dead_cat_bounce = indicator[(indicator['future_dead_cat_bounce'] == True)]

  figure_stock.add_trace(
      go.Scatter(
          mode='markers',
          x=future_dead_cat_bounce.index,
          y=future_dead_cat_bounce['Open'],
          marker_symbol='triangle-down',
          marker=dict(
              color='Green',
              size=20,
              opacity=0.5,
              line=dict(
                  color='Blue',
                  width=2)),
          showlegend=True,
          name='Dead Cat Bounce With Prediction'))

  # push will be triggered if a Dead Cat Bounce happens "today"
  if (dt.datetime.today() in future_dead_cat_bounce.index):
    send_push('Watch out, you might run into a Dead Cat Bounce!')

In [ ]:
predict_falling_market()

# **Plot Graphs**

In [ ]:
# Plot graph including dead cat bounce but without prediction
plot_dead_cat_bounce()

# Plot graphs in total
figure_stock.show()
figure_macd.show()
figure_prediction.show()